In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

# Utilizando un modelo pre-entrenado

[`torchvision.models`](https://pytorch.org/docs/stable/torchvision/models.html) ofrece una serie de modelos famosos de la literatura de *deep learning*

Por defecto el modelo se carga con pesos aleatorios

Si indicamos `pretrained=True` se descarga un modelo entrenado

Se pueden escoger modelos para clasificar, localizar y segmentar

## Modelo para clasificar imágenes

torchvision tiene una basta cantidad de modelos para clasificar incluyendo distintas versiones de VGG, ResNet, AlexNet, GoogLeNet, DenseNet, entre otros

Cargaremos un modelo [resnet18](https://arxiv.org/pdf/1512.03385.pdf) [pre-entrenado](https://pytorch.org/docs/stable/torchvision/models.html#torchvision.models.resnet18) en [ImageNet](http://image-net.org/) 

In [ ]:
from torchvision import models

model = models.resnet18(pretrained=True, progress=True)
model.eval()

Los modelos pre-entrenados esperan imágenes con
- tres canales (RGB)
- al menos 224x224 píxeles
- píxeles entre 0 y 1 (float)
- normalizadas con 

        normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])



In [ ]:
from PIL import Image
import torch
from torchvision import transforms

img = Image.open("img/dog.jpg")

my_transform = transforms.Compose([transforms.Resize(256),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=(0.485, 0.456, 0.406), 
                                                        std=(0.229, 0.224, 0.225))])

# Las clases con probabilidad más alta son
probs = torch.nn.Softmax(dim=1)(model.forward(my_transform(img).unsqueeze(0)))

best = probs.argsort(descending=True)
display(best[0, :5], probs[0, best[0, :5]])

¿A qué corresponde estas clases?

Clases de ImageNet: https://gist.github.com/ageitgey/4e1342c10a71981d0b491e1b8227328b


## Modelo para detectar entidades en imágenes

Adicional a los modelos de clasificación torchvision también tiene modelos para
- Detectar entidades en una imagen: Faster RCNN
- Hacer segmentación por instancia: Mask RCNN
- Hacer segmentación semántica: FCC, DeepLab
- Clasificación de video 

A continuación probaremos la [Faster RCNN](https://arxiv.org/abs/1506.01497) para hace detección

Este modelo fue pre-entrenado en la base de datos [COCO](https://cocodataset.org/)

El modelo retorna un diccionario con
- 'boxes': Los bounding box de las entidades
- 'labels': La etiqueta de la clase más probable de la entidad
- 'score': La probabilidad de la etiqueta

In [ ]:
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

transform = transforms.ToTensor()
img = Image.open("img/pelea.jpg") # No require normalización de color
img_tensor = transform(img)

result = model(img_tensor.unsqueeze(0))[0]

def filter_results(result, threshold=0.9):
    mask = result['scores'] > 0.9
    bbox = result['boxes'][mask].detach().cpu().numpy()
    lbls = result['labels'][mask].detach().cpu().numpy()
    return bbox, lbls

In [ ]:
from PIL import ImageFont, ImageDraw
fnt = ImageFont.truetype("arial.ttf", 20) 

label2name = {1: 'persona', 2: 'bicicleta', 3: 'auto', 4: 'moto', 
              8: 'camioneta', 18: 'perro'}

def draw_rectangles(img, bbox, lbls):
    draw = ImageDraw.Draw(img)
    for k in range(len(bbox)):
        if lbls[k] in label2name.keys():
            draw.rectangle(bbox[k], fill=None, outline='white', width=2)
            draw.text([int(d) for d in bbox[k][:2]], label2name[lbls[k]], font=fnt, fill='white')

bbox, lbls = filter_results(result)
img = Image.open("img/pelea.jpg")
draw_rectangles(img, bbox, lbls)
display(img)

# Transferencia de Aprendizaje


A continuación usaremos la técnicas de transferencia de aprendizaje para aprender un clasificador de imágenes para un fragmento de la base de datos food 5k

El objetivo es clasificar si la imagen corresponde a comida o no

Guardamos las imagenes con la siguiente estructura de carpetas

In [ ]:
!ls img/food5k/
!ls img/food5k/train

Con esto podemos usar `torchvision.datasets.ImageFolder` para crear los dataset de forma muy sencilla

Dado que usaremos un modelo preentrenado debemos transformar entregar las imágenes en tamaño 224x224 y con color normalizado

Usaremos también aumentación de datos en el conjunto de entrenamiento

In [ ]:
from torchvision import datasets

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

train_dataset = datasets.ImageFolder('img/food5k/train', transform=train_transforms)
valid_dataset = datasets.ImageFolder('img/food5k/valid', transform=valid_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=256, shuffle=False)

for image, label in train_loader:
    break
    
fig, ax = plt.subplots(1, 6, figsize=(9, 2), tight_layout=True)
for i in range(6):
    ax[i].imshow(image[i].permute(1,2,0).numpy())
    ax[i].axis('off')
    ax[i].set_title(label[i].numpy())

Usaremos el modelo ResNet18


In [ ]:
model = models.resnet18(pretrained=True, progress=True)
display(model)

En este caso re-entrenaremos sólo la última capa: `fc`

Las demás capas las congelaremos

Para congelar una capa simplemente usamos `requires_grad=False` en sus parámetros

Cuando llamemos `backward` no se calculará gradiente para estas capas

In [ ]:
#Congelamos todos los parámetros
for param in model.parameters(): 
    param.requires_grad = False

# Recuperamos el número de neuronas de la última capa
neurons = model.fc.in_features 
# La reemplazamos por una nueva capa de salida
model.fc = torch.nn.Linear(neurons, 2) 

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    for x, y in train_loader:
        optimizer.zero_grad()
        yhat = model.forward(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()

    epoch_loss = 0.0
    for x, y in valid_loader:
        yhat = model.forward(x)
        loss = criterion(yhat, y)
        epoch_loss += loss.item()
    print(epoch, epoch_loss)

In [ ]:
targets, predictions = [], []
for mbdata, label in valid_loader:
    logits = model.forward(mbdata)
    predictions.append(logits.argmax(dim=1).detach().numpy())
    targets.append(label.numpy())
predictions = np.concatenate(predictions)
targets = np.concatenate(targets)

from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(targets, predictions)
display(cm)

print(classification_report(targets, predictions))

# Resumen

Aspectos a considerar durante el entrenamiento de redes neuronales
- Arquitecturas: cantidad y organización de capas, funciones de activación
- Funciones de costo, optimizadores y sus parámetros (tasa de aprendizaje, momentum)
- Verificar convergencia y sobreajuste:
    - Checkpoint: Guardar el último modelo y el con menor costo de validación
    - Early stopping: Detener el entrenamiento si el error de validación no disminuye en un cierto número de épocas
- Inicialización de los parámetros: Probar varios entrenamientos desde inicios aleatorios distintos
- Si el modelo se sobreajusta pronto
    - Disminuir complejidad
    - Regularizar: Aumentación de datos, decaimiento de pesos, Dropout
- Si quiero aprovechar un modelo preentrenado
    - Transferencia de aprendizaje
    - [Zoológico de modelos](https://modelzoo.co/)

Estrategia agil
> Desarrolla rápido e itera: Empieza simple. Propón una solución, impleméntala, entrena y evalua. Analiza las fallas, modifica e intenta de nuevo

Mucho exito en sus desarrollos futuros!